![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_RoBertaForSequenceClassification.ipynb)

# Import OpenVINO RoBertaForSequenceClassification models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting RoBertaForSequenceClassification models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for RoBertaForSequenceClassification from RoBertaForSequenceClassification   and they have to be in `Text Classification` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers==4.48.3 optimum[openvino]

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [deepset/roberta-base-squad2](https://huggingface.co/deepset/roberta-base-squad2) model from HuggingFace as an example and load it as a `OVModelForQuestionAnswering`, representing an OpenVINO model.
- In addition to the OVModelForQuestionAnswering model, we also need to save the `AutoTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from optimum.intel import OVModelForSequenceClassification
from transformers import AutoTokenizer

MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment-latest"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

ov_model = OVModelForSequenceClassification.from_pretrained(MODEL_NAME, export=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

ov_model.save_pretrained(EXPORT_PATH)
tokenizer.save_pretrained(EXPORT_PATH)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


('ov_models/cardiffnlp/twitter-roberta-base-sentiment-latest/tokenizer_config.json',
 'ov_models/cardiffnlp/twitter-roberta-base-sentiment-latest/special_tokens_map.json',
 'ov_models/cardiffnlp/twitter-roberta-base-sentiment-latest/vocab.json',
 'ov_models/cardiffnlp/twitter-roberta-base-sentiment-latest/merges.txt',
 'ov_models/cardiffnlp/twitter-roberta-base-sentiment-latest/added_tokens.json',
 'ov_models/cardiffnlp/twitter-roberta-base-sentiment-latest/tokenizer.json')

Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -lR {EXPORT_PATH}

- We need to convert `vocab.json` to a plain `vocab.txt` format, as required by Spark NLP.
- Move both `vocab.txt` and `merges.txt` into the assets folder.
- Additionally, we need to extract label-to-ID mappings from the model config and save them as `labels.txt` in the same folder for Spark NLP to use during inference.

In [5]:
import json

!mkdir -p {EXPORT_PATH}/assets && mv {EXPORT_PATH}/merges.txt {EXPORT_PATH}/assets/

with open(f"{EXPORT_PATH}/vocab.json") as f, open(f"{EXPORT_PATH}/assets/vocab.txt", "w") as out:
    out.write("\n".join(json.load(f)))

with open(f"{EXPORT_PATH}/assets/labels.txt", "w") as f:
    f.write("\n".join(ov_model.config.id2label[k] for k in sorted(ov_model.config.id2label)))

In [6]:
!ls -lR {EXPORT_PATH}

ov_models/cardiffnlp/twitter-roberta-base-sentiment-latest:
total 491560
drwxr-xr-x 2 root root      4096 Jun 24 12:03 assets
-rw-r--r-- 1 root root       915 Jun 24 12:03 config.json
-rw-r--r-- 1 root root 498596028 Jun 24 12:03 openvino_model.bin
-rw-r--r-- 1 root root    377768 Jun 24 12:03 openvino_model.xml
-rw-r--r-- 1 root root       958 Jun 24 12:03 special_tokens_map.json
-rw-r--r-- 1 root root      1274 Jun 24 12:03 tokenizer_config.json
-rw-r--r-- 1 root root   3558643 Jun 24 12:03 tokenizer.json
-rw-r--r-- 1 root root    798293 Jun 24 12:03 vocab.json

ov_models/cardiffnlp/twitter-roberta-base-sentiment-latest/assets:
total 852
-rw-r--r-- 1 root root     25 Jun 24 12:03 labels.txt
-rw-r--r-- 1 root root 456318 Jun 24 12:03 merges.txt
-rw-r--r-- 1 root root 407064 Jun 24 12:03 vocab.txt


In [7]:
!cat {EXPORT_PATH}/assets/labels.txt

negative
neutral
positive

Voila! We have our `vocab.txt`, `merges.txt` and `labels.txt` inside assets directory

## Import and Save RoBertaForSequenceClassification in Spark NLP


- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [8]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 24.9 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [9]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.5.3
Apache Spark version: 3.5.4


- Let's use `loadSavedModel` functon in `RoBertaForSequenceClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `RoBertaForSequenceClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [10]:
from sparknlp.annotator import RoBertaForSequenceClassification

sequenceClassifier = RoBertaForSequenceClassification.loadSavedModel(
     EXPORT_PATH,
     spark
 )\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [11]:
sequenceClassifier.write().overwrite().save("./{}_spark_nlp_openvino".format(EXPORT_PATH))

Let's clean up stuff we don't need anymore

In [12]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your RoBertaForSequenceClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [13]:
! ls -l {EXPORT_PATH}_spark_nlp_openvino

total 487368
drwxr-xr-x 5 root root      4096 Jun 24 12:06 fields
drwxr-xr-x 2 root root      4096 Jun 24 12:06 metadata
-rw-r--r-- 1 root root 499050214 Jun 24 12:06 roberta_classification_openvino


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny RoBertaForSequenceClassification model 😊

In [14]:
sequenceClassifier_loaded = RoBertaForSequenceClassification.load("./{}_spark_nlp_openvino".format(EXPORT_PATH))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

You can see what labels were used to train this model via `getClasses` function:

In [15]:
sequenceClassifier_loaded.getClasses()

['negative', 'neutral', 'positive']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [16]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    sequenceClassifier_loaded
])

data = spark.createDataFrame([
    ["I love you!"],
    ["Kill yourself"]
], ["text"])

model = pipeline.fit(data)
result = model.transform(data)

result.select("text", "class.result").show(truncate=False)

+-------------+----------+
|text         |result    |
+-------------+----------+
|I love you!  |[positive]|
|Kill yourself|[negative]|
+-------------+----------+



That's it! You can now go wild and use hundreds of `RoBertaForSequenceClassification` models from HuggingFace 🤗 in Spark NLP 🚀
